In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import os,warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from tabulate import tabulate
import datetime

In [ ]:
# paths
TRAIN_PATH = '/content/train.csv'
TEST_PATH = '/content/test.csv'
SUBMISSION_PATH = '/content/sample_submission (1).csv'

# read data
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
submit_df = pd.read_csv(SUBMISSION_PATH)

# drop column = id
test_df.drop('ID',axis=1,inplace=True)

# drop NULLs
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

# reset index
train_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

# view
print(f"Training data shape: {train_df.shape}")

Training data shape: (400000, 31)


In [ ]:
# feature engineering
X = train_df.iloc[:, : 30]
Y = train_df[['Target']]


# Standardize features
scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)
# split data
#x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0)
#print(f"x_train shape: {x_train.shape} | x_test shape: {x_test.shape}")

In [ ]:
%%time

# import the library
from sklearn.svm import LinearSVC

# instantiate & fit
svm=LinearSVC(C=0.0001)
svm.fit(X, Y)
data_test = pd.read_csv("/content/test.csv")

# Extract features for testing
x_test = scaler.transform(data_test.drop(columns=['ID'], axis=1))

CPU times: user 4 s, sys: 176 ms, total: 4.18 s
Wall time: 4.18 s


In [ ]:
y_test = model.predict(x_test)
binary_predictions = (y_test > 0.5).astype(int)

3125/3125 [==============================] - 7s 2ms/step


In [ ]:
submission_df = pd.DataFrame({'ID': data_test['ID'].values.flatten(), 'Target': binary_predictions.flatten()})


# Save the submission DataFrame to a CSV file
submission_df.to_csv("submission6.csv", index=False)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

#numpy array
#X_array = X.values

#reshaping the data
#X_array = X_array.reshape(X_array.shape[0], X_array.shape[1], 1)

# Split the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X_array, Y, random_state=42)
scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)



model = Sequential()
model.add(Dense(64, input_shape=(X.shape[-1],), activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.summary()

metrics = [
    keras.metrics.BinaryAccuracy(name="accuracy"),
]

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(0.01),
             loss="binary_crossentropy",
             metrics=metrics)
epochs = 10
batch_size = 128

# Training the model
history = model.fit(X, Y, epochs=10)


predictions = model.predict(x_test)

# Optionally, if you want binary predictions (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Accuracy
accuracy = history.history['accuracy']


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 64)                1984      
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dense_15 (Dense)            (None, 16)                528       
                                                                 
 dense_16 (Dense)            (None, 1)                 17        
                                                                 
Total params: 4609 (18.00 KB)
Trainable params: 4609 (18.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
12500/12500 [==============================] - 42s 3ms/step - loss: 0.6430 - accuracy: 0.6187
Epoch 2/10
12500/12500 [=======================

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
def model_builder(hp):
  model = keras.Sequential()

  # Tuning the number of units in the first Dense layer (32-512)

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))

  # Tuning the learning rate for the optimizer (0.001, 0.0001, or 0.00001)

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])

  metrics = [
    keras.metrics.BinaryAccuracy("accuracy"),
    ]

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.BinaryCrossentropy(),
                metrics=metrics)

  return model

In [ ]:
tuner = kt.RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
tuner.search(X, Y, epochs=10, callbacks=[stop_early], batch_size=64)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=3)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
96                |96                |units
0.0001            |0.0001            |learning_rate

Epoch 1/10
6234/6250 [============================>.] - ETA: 0s - loss: 0.6700 - accuracy: 0.5838

6250/6250 [==============================] - 18s 3ms/step - loss: 0.6699 - accuracy: 0.5839
Epoch 2/10
6247/6250 [============================>.] - ETA: 0s - loss: 0.6440 - accuracy: 0.6252

6250/6250 [==============================] - 16s 3ms/step - loss: 0.6440 - accuracy: 0.6252
Epoch 3/10
6235/6250 [============================>.] - ETA: 0s - loss: 0.6369 - accuracy: 0.6360

6250/6250 [==============================] - 16s 3ms/step - loss: 0.6369 - accuracy: 0.6360
Epoch 4/10
6230/6250 [============================>.] - ETA: 0s - loss: 0.6330 - accuracy: 0.6416

6250/6250 [==============================] - 16s 3ms/step - loss: 0.6330 - accuracy: 0.6416
Epoch 5/10
6248/6250 [============================>.] - ETA: 0s - loss: 0.6298 - accuracy: 0.6461

...

6250/6250 [==============================] - 18s 3ms/step - loss: 0.6212 - accuracy: 0.6568
Epoch 9/10
6236/6250 [============================>.] - ETA: 0s - loss: 0.6185 - accuracy: 0.6601

6250/6250 [==============================] - 17s 3ms/step - loss: 0.6185 - accuracy: 0.6602
Epoch 10/10
6234/6250 [============================>.] - ETA: 0s - loss: 0.6158 - accuracy: 0.6624

6250/6250 [==============================] - 17s 3ms/step - loss: 0.6157 - accuracy: 0.6625


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 264, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner_utils.py", line 132, in <listcomp>
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner_utils.py", line 145, in convert_to_metrics_dict
    best_value, _ = _get_best_value_and_best_epoch_from_history(
  File "/usr/local/lib/python3.10/dist-packages/kera

RuntimeError: ignored

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation




best_hps = {'units': 96, 'learning_rate': 0.0001}

# Building the model with the best hyperparameters
model = keras.Sequential()
model.add(keras.layers.Dense(units=best_hps['units'], input_shape=(X.shape[-1],), activation='relu'))
model.add(keras.layers.Dense(units=best_hps['units'], activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compiling the model with the best learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate=best_hps['learning_rate']),
              loss='binary_crossentropy',
              metrics=['accuracy'])

#summary
model.summary()
model.save('Higgs Boson.h5')
# Training the model
epochs = 10
batch_size = 64
history = model.fit(X, Y, epochs=epochs, batch_size=batch_size)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 96)                2976      
                                                                 
 dense_4 (Dense)             (None, 96)                9312      
                                                                 
 dense_5 (Dense)             (None, 1)                 97        
                                                                 
Total params: 12385 (48.38 KB)
Trainable params: 12385 (48.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
6250/6250 [==============================] - 17s 3ms/step - loss: 0.6697 - accuracy: 0.5836
Epoch 2/10
6250/6250 [==============================] - 16s 3ms/step - loss: 0.6423 - accuracy: 0.6265
Epoch 3/10
6250/6250 [==============================] - 17s 3ms/step - loss:

In [ ]:
predictions = model.predict(x_test)
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)

# DataFrame with predictions_df
predictions_df = pd.DataFrame()
#predictions_df['Actual'] = y_test.iloc[:, 0]
predictions_df = binary_predictions[:, 0]

3125/3125 [==============================] - 6s 2ms/step


In [ ]:
submission_df = pd.DataFrame()
submission_df['ID'] = range(1, len(predictions_df) + 1)  # Serial numbers starting from 1
submission_df['Target'] = predictions_df.astype(int)  # Answer' is of integer type

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission10.csv', index=False)

# Print the submission DataFrame
print(submission_df)

           ID  Target
0           1       1
1           2       0
2           3       1
3           4       1
4           5       0
...       ...     ...
99995   99996       0
99996   99997       0
99997   99998       1
99998   99999       1
99999  100000       1

[100000 rows x 2 columns]


In [ ]:
df_temp = pd.read_csv('submission10.csv')
test_temp = pd.read_csv('test.csv')
assert len(df_temp.columns) == 2, "Number of columns in the submission file is not correct, check the submission format"
assert list(df_temp.columns) == ['ID', 'Target'] , "Column names are not correct, check the submission format"
assert df_temp['Target'].nunique() == 1 or df_temp['Target'].nunique() == 2, "The prediction should be 0 or 1 only"
assert len(df_temp) == len(test_temp), "Number of rows in the submission file is not correct"